In [1]:
import os
import re
import csv
import torch
import tifffile
import numpy as np
from scipy.ndimage import zoom
from scipy.ndimage import binary_dilation

In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [47]:
zoom_factor = 1 / 5.8824    # 5.8824 is the z scaling factor applied before for converting to isotropic voxel

def process_images(base_folder, mask_folder, output_folder, results_folder):
    intensity_values = []

    # Name output file for measurements of all images in the folder
    output_file = os.path.join(results_folder, 'neuron_measurements.csv')
    
    pattern = re.compile(r'cell(\d+)')

    for i in range(0, 41):  # 41 folder for 41 fields of view
        folder_name = f"z{i}"
        print("Processing folder:", folder_name)
        image_folder = os.path.join(base_folder, folder_name)
        mask_image_path = os.path.join(mask_folder, f"z{i}.tif")

        # Load and transfer mask to GPU
        mask_image = tifffile.imread(mask_image_path).astype(np.uint16)  # Convert to uint16
        mask_image = torch.tensor(mask_image, dtype=torch.float32, device=device)
        mask_rescaled = torch.tensor(zoom(mask_image.cpu().numpy(), (zoom_factor, 1, 1), order=0), device=device)

        img_dir = "./Bigstream_notebook/"
        GFAP_img_tif = os.path.join(img_dir, f"Aligned_zstack{i}_Time5_channel0.tiff")
        gfap_image = tifffile.imread(GFAP_img_tif).astype(np.uint16)
        gfap_image = torch.tensor(gfap_image, dtype=torch.float32, device=device)

        output_z_file = os.path.join(results_folder, f"z{i}.csv")
        intensity_values_z = []

        for filename in os.listdir(image_folder):
            if 't0_c0' in filename and filename.endswith('.tiff'):
                try:
                    cell_number = int(filename.split('cell')[1].split('.tiff')[0])
                except (IndexError, ValueError):
                    print(f"Skipping file {filename} due to incorrect format.")
                    continue

                # Find the mask with the cell number
                cell_mask = (mask_rescaled == cell_number).to(torch.uint8) # * 255

                if torch.any(cell_mask):
                    # Measure the 3D size of the mask
                    non_zero_z = torch.sum(torch.any(cell_mask > 0, dim=(1, 2))).item()
                    non_zero_y = torch.sum(torch.any(cell_mask > 0, dim=(0, 2))).item()
                    non_zero_x = torch.sum(torch.any(cell_mask > 0, dim=(0, 1))).item()

                    # Find middle plane coordinates
                    coords = torch.nonzero(cell_mask > 0, as_tuple=False)
                    min_z, min_y, min_x = torch.min(coords, dim=0).values.cpu().numpy()
                    max_z, max_y, max_x = torch.max(coords, dim=0).values.cpu().numpy()

                    mid_z, mid_y, mid_x = (min_z + max_z) // 2, (min_y + max_y) // 2, (min_x + max_x) // 2

                    gfap_slice = gfap_image[mid_z, :, :]
                    gfap_slice_tensor = gfap_slice.to(dtype=torch.float32).detach()

                    # Create and expand the middle plane mask
                    middle_plane_mask = cell_mask[mid_z, :, :].cpu().numpy().astype(np.uint8)
                    expanded_mask = binary_dilation(middle_plane_mask, iterations=5)

                    # exclusive_expanded_mask = expanded_mask & ~cell_mask
                    exclusive_mask = expanded_mask.astype(np.int8) - middle_plane_mask.astype(np.uint8)
                    exclusive_mask = torch.tensor(exclusive_mask, dtype=torch.uint8, device=device)
                    
                    # Measure integrated intensity of GFAP in the 5-pixel ring around nucleus mid-plane
                    gfap_intensity_exclusive = torch.mean(gfap_slice_tensor[exclusive_mask > 0]).item()

                    # Store results
                    intensity_values.append([
                        folder_name, cell_number, gfap_intensity_exclusive
                    ])
                    intensity_values_z.append([
                        folder_name, cell_number, gfap_intensity_exclusive
                    ])
                    # print("Processed:", folder_name, cell_number, gfap_intensity_exclusive)

        
        # Save per folder results
        with open(output_z_file, mode='w', newline='') as file_z:
            writer = csv.writer(file_z)
            writer.writerow(['Folder name', 'Cell Number', 'GFAP_intensity_exclusive'])
            writer.writerows(intensity_values_z)
                                    
    # Save final results to CSV
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Folder name', 'Cell Number', 'GFAP_intensity_exclusive'])
        writer.writerows(intensity_values)

if __name__ == "__main__":
    base_folder = "./segmented_neurons/"       # repeat for astrocyte folder
    mask_folder = "./CP_masks4/"
    output_folder = r"E:\Neuron_17Ab_analysis\cell_measurements\GFAP"
    results_folder = r"E:\Neuron_17Ab_analysis\cell_measurements\GFAP\neuron"

    process_images(base_folder, mask_folder, output_folder, results_folder)

Processing folder: z0
Processing folder: z1
Processing folder: z2
Processing folder: z3
Processing folder: z4
Processing folder: z5
Processing folder: z6
Processing folder: z7
Processing folder: z8
Processing folder: z9
Processing folder: z10
Processing folder: z11
Processing folder: z12
Processing folder: z13
Processing folder: z14
Processing folder: z15
Processing folder: z16
Processing folder: z17
Processing folder: z18
Processing folder: z19
Processing folder: z20
Processing folder: z21
Processing folder: z22
Processing folder: z23
Processing folder: z24
Processing folder: z25
Processing folder: z26
Processing folder: z27
Processing folder: z28
Processing folder: z29
Processing folder: z30
Processing folder: z31
Processing folder: z32
Processing folder: z33
Processing folder: z34
Processing folder: z35
Processing folder: z36
Processing folder: z37
Processing folder: z38
Processing folder: z39
Processing folder: z40
